## Import

In [0]:
#pip install tensorflow
#pip install plotly
#pip install bs4
#pip install keras
#pip install lxml

In [0]:
import warnings
warnings.filterwarnings('ignore')
import re
from io import StringIO
from datetime import datetime, timedelta
import requests
import math
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots  
from plotly.offline import iplot
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import json
import csv
from bs4 import BeautifulSoup
import requests, lxml
from lxml import html
from io import StringIO
import pyspark
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import round

## Données historiques

On va webscrapper les données historiques de plusieurs valeurs technologiques, il s'agit de 'Apple Inc - AAPL', 'Tesla Inc - TSLA', 'Microsoft Corporation - MSFT', 
                                                                                      'Amazon.com Inc - AMZN', 'Alphabet Inc - GOOGL',
                                                                                  'Meta Platforms Inc - FB' et 'NVIDIA Corporation - NVDA'

In [0]:
class WebScrapStock:
    
    timeout = 30
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    stock_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


    def __init__(self, stock_name, days_back=7):
        self.stock_name = stock_name
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.stock_name), timeout=self.timeout, verify = False, headers=self.headers_)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        
        if not match:
            raise ValueError("Pas de réponse de Yahoo Finance")
        else:
            self.crumb = match.group(1)

    def get_stock(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.stock_link.format(quote=self.stock_name, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url, verify = False, headers=self.headers_)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text), parse_dates=['Date'])
        df["Stock"] = self.stock_name
        df = df[["Date", "Stock", "Open", "High", "Low", "Close", "Adj Close", "Volume"	]]
        return df, url

In [0]:

#On récupère les données historiques des différentes actions pour 10 années
jours = 360*10
df1, url1 = WebScrapStock(stock_name  = 'AAPL', days_back=jours).get_stock()
df2, url2 = WebScrapStock(stock_name  = 'TSLA', days_back=jours).get_stock()
df3, url3 = WebScrapStock(stock_name  = 'MSFT', days_back=jours).get_stock()
df4, url4 = WebScrapStock(stock_name  = 'AMZN', days_back=jours).get_stock()
df5, url5 = WebScrapStock(stock_name  = 'GOOGL', days_back=jours).get_stock()
df6, url6 = WebScrapStock(stock_name  = 'FB', days_back=jours).get_stock()
df7, url7 = WebScrapStock(stock_name  = 'NVDA', days_back=jours).get_stock()

df8 = df1.append(df2)
df9 = df8.append(df3)
df10 = df9.append(df4)
df11 = df10.append(df5)
df12 = df11.append(df6)
df13 = df11.append(df7)


data_ = df13.copy()
data_

,Date,Stock,Open,High,Low,Close,Adj Close,Volume
0,2012-02-17,AAPL,17.968214,18.134644,17.867857,17.932858,15.375009,535805200
1,2012-02-21,AAPL,18.102858,18.387501,18.004286,18.387501,15.764801,605595200
2,2012-02-22,AAPL,18.324286,18.410357,18.181070,18.322857,15.709375,483302400
3,2012-02-23,AAPL,18.395714,18.493929,18.196428,18.442499,15.811956,568027600
4,2012-02-24,AAPL,18.559643,18.674999,18.522858,18.657499,15.996286,415072000
...,...,...,...,...,...,...,...,...
2475,2021-12-17,NVDA,279.850006,289.220001,277.600006,278.010010,278.010010,71375800
2476,2021-12-20,NVDA,273.049988,281.440002,271.450012,277.190002,277.190002,46184700
2477,2021-12-21,NVDA,283.739990,291.200012,274.010010,290.750000,290.750000,52438500
2478,2021-12-22,NVDA,288.910004,295.549988,284.489990,294.000000,294.000000,39518400


In [0]:

fig = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(
x = df2["Date"],
y = df2["Close"],
mode = "lines",
name = "AAPL ",
marker = dict(color = 'grey'))

trace2 = go.Scatter(
x = df2["Date"],
y = df2["Close"],
mode = "lines",
name = "TSLA",
marker = dict(color = 'black'))

trace3 = go.Scatter(
x = df3["Date"],
y = df3["Close"],
mode = "lines",
name = "MSFT ",
marker = dict(color = 'red'))


trace4 = go.Scatter(
x = df4["Date"],
y = df4["Close"],
mode = "lines",
name = "AMZN",
marker = dict(color = 'green'))

trace5 = go.Scatter(
x = df5["Date"],
y = df5["Close"],
mode = "lines",
name = "GOOGL",
marker = dict(color = 'yellow'))


trace6 = go.Scatter(
x = df6["Date"],
y = df6["Close"],
mode = "lines",
name = "FB",
marker = dict(color = 'blue'))


trace7 = go.Scatter(
x = df7["Date"],
y = df7["Close"],
mode = "lines",
name = "NVDA",
marker = dict(color = 'pink'))


fig.add_trace(trace1, secondary_y=True)
fig.add_trace(trace2, secondary_y=True)
fig.add_trace(trace3, secondary_y=True)
fig.add_trace(trace4, secondary_y=True)
fig.add_trace(trace5, secondary_y=True)
fig.add_trace(trace6, secondary_y=True)
fig.add_trace(trace7, secondary_y=True)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='Red', rangeslider_visible=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black');
data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(autosize=False,
                  width=1000,
                  height=550,
                  title = "Évolution du cours de l'action depuis " + str(data_.iloc[0, 0])[:10],
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de fermetures",ticklen= 5, zeroline= True, showline=True, showgrid=False), plot_bgcolor='#404754')
fig = dict(data = data, layout = layout)
#fig.update_xaxes()
iplot(fig)

On voit que le titre AMZN a un coût plus élevé que les autres. On remarque aussi une baisse de tous les cours vers 2020 après une ralentissment des marchés financiers. Ensuite s'en suit un décollage, coincidant avec la hausse des valeurs technologiques.

## Data engineering avec Spark

In [0]:
df_spark=spark.createDataFrame(data_)
print("La base de données contient", df_spark.count(), "observations et", len(df_spark.columns), "variables.")

df_spark = df_spark.orderBy(['Stock', 'Date'],
                          ascending=False)
df_spark.show()

La base de données contient 14880 observations et 8 variables.
+-------------------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|               Date|Stock|       Open|       High|        Low|      Close|  Adj Close|  Volume|
+-------------------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|2021-12-23 00:00:00| TSLA|1006.799988| 1072.97998| 997.559998|     1067.0|     1067.0|30841900|
|2021-12-22 00:00:00| TSLA| 965.659973|1015.659973| 957.049988|1008.869995|1008.869995|31211400|
|2021-12-21 00:00:00| TSLA| 916.869995|      939.5| 886.119995| 938.530029| 938.530029|23839300|
|2021-12-20 00:00:00| TSLA| 910.700012| 921.690002| 893.390015| 899.940002| 899.940002|18826700|
|2021-12-17 00:00:00| TSLA|  914.77002| 960.659973| 909.039978| 932.570007| 932.570007|33479100|
|2021-12-16 00:00:00| TSLA|      994.5|  994.97998| 921.849976| 926.919983| 926.919983|27590500|
|2021-12-15 00:00:00| TSLA| 953.210022|     978.75|     928.25| 

In [0]:
#On formatte la date pour enlever les heures et minutes
df_p = pyspark.sql.functions.split(df_spark['Date'], '\s')
df_spark= df_spark.withColumn('Date',df_p.getItem(0))
df_spark.show()

+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|      Date|Stock|       Open|       High|        Low|      Close|  Adj Close|  Volume|
+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|2021-12-23| TSLA|1006.799988| 1072.97998| 997.559998|     1067.0|     1067.0|30841900|
|2021-12-22| TSLA| 965.659973|1015.659973| 957.049988|1008.869995|1008.869995|31211400|
|2021-12-21| TSLA| 916.869995|      939.5| 886.119995| 938.530029| 938.530029|23839300|
|2021-12-20| TSLA| 910.700012| 921.690002| 893.390015| 899.940002| 899.940002|18826700|
|2021-12-17| TSLA|  914.77002| 960.659973| 909.039978| 932.570007| 932.570007|33479100|
|2021-12-16| TSLA|      994.5|  994.97998| 921.849976| 926.919983| 926.919983|27590500|
|2021-12-15| TSLA| 953.210022|     978.75|     928.25|  975.98999|  975.98999|25056400|
|2021-12-14| TSLA|      945.0| 966.409973|      930.0|  958.51001|  958.51001|23602100|
|2021-12-13| TSLA|1001.090027|  

In [0]:
# On créée un index pour pouvoir déterminer un taux de variation des valeurs de fermeture
df_spark = df_spark.withColumn("id", monotonically_increasing_id())

In [0]:
my_window = Window.partitionBy().orderBy("id")

#on dréée d'abord 2 colonnes prev et diff qui serviront à calculer les taux
df_spark = df_spark.withColumn("prev_value", F.lag(df_spark.Close).over(my_window))

df_spark = df_spark.withColumn("diff", F.when(F.isnull(df_spark.Close - df_spark.prev_value), 0)
                              .otherwise(df_spark.Close - df_spark.prev_value))

df_spark = df_spark.withColumn('Variations %', F.col('diff') / F.col('Close'))
df_spark.show()

+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+---+-----------+-------------------+--------------------+
|      Date|Stock|       Open|       High|        Low|      Close|  Adj Close|  Volume| id| prev_value|               diff|        Variations %|
+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+---+-----------+-------------------+--------------------+
|2021-12-23| TSLA|1006.799988| 1072.97998| 997.559998|     1067.0|     1067.0|30841900|  0|       null|                0.0|                 0.0|
|2021-12-22| TSLA| 965.659973|1015.659973| 957.049988|1008.869995|1008.869995|31211400|  1|     1067.0| -58.13000499999998|-0.05761892541962256|
|2021-12-21| TSLA| 916.869995|      939.5| 886.119995| 938.530029| 938.530029|23839300|  2|1008.869995|         -70.339966|-0.07494695302924613|
|2021-12-20| TSLA| 910.700012| 921.690002| 893.390015| 899.940002| 899.940002|18826700|  3| 938.530029|-38.590026999999964|-0.0428

In [0]:
to_round = ['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'id',
 'prev_value',
 'diff',
 'Variations %']

for c in to_round:
    df_spark = df_spark.withColumn(c, round(c, 3))


df_spark.show()

+----------+-----+-------+-------+-------+-------+---------+--------+---+----------+------+------------+
|      Date|Stock|   Open|   High|    Low|  Close|Adj Close|  Volume| id|prev_value|  diff|Variations %|
+----------+-----+-------+-------+-------+-------+---------+--------+---+----------+------+------------+
|2021-12-23| TSLA| 1006.8|1072.98| 997.56| 1067.0|   1067.0|30841900|  0|      null|   0.0|         0.0|
|2021-12-22| TSLA| 965.66|1015.66| 957.05|1008.87|  1008.87|31211400|  1|    1067.0|-58.13|      -0.058|
|2021-12-21| TSLA| 916.87|  939.5| 886.12| 938.53|   938.53|23839300|  2|   1008.87|-70.34|      -0.075|
|2021-12-20| TSLA|  910.7| 921.69| 893.39| 899.94|   899.94|18826700|  3|    938.53|-38.59|      -0.043|
|2021-12-17| TSLA| 914.77| 960.66| 909.04| 932.57|   932.57|33479100|  4|    899.94| 32.63|       0.035|
|2021-12-16| TSLA|  994.5| 994.98| 921.85| 926.92|   926.92|27590500|  5|    932.57| -5.65|      -0.006|
|2021-12-15| TSLA| 953.21| 978.75| 928.25| 975.99|   97

In [0]:
df_spark = df_spark.select(
    ["Date",'Stock', 'Open', 'High', 'Close', 'Adj Close', 'Variations %']
                )
df_spark.show()

+----------+-----+-------+-------+-------+---------+------------+
|      Date|Stock|   Open|   High|  Close|Adj Close|Variations %|
+----------+-----+-------+-------+-------+---------+------------+
|2021-12-23| TSLA| 1006.8|1072.98| 1067.0|   1067.0|         0.0|
|2021-12-22| TSLA| 965.66|1015.66|1008.87|  1008.87|      -0.058|
|2021-12-21| TSLA| 916.87|  939.5| 938.53|   938.53|      -0.075|
|2021-12-20| TSLA|  910.7| 921.69| 899.94|   899.94|      -0.043|
|2021-12-17| TSLA| 914.77| 960.66| 932.57|   932.57|       0.035|
|2021-12-16| TSLA|  994.5| 994.98| 926.92|   926.92|      -0.006|
|2021-12-15| TSLA| 953.21| 978.75| 975.99|   975.99|        0.05|
|2021-12-14| TSLA|  945.0| 966.41| 958.51|   958.51|      -0.018|
|2021-12-13| TSLA|1001.09| 1005.0| 966.41|   966.41|       0.008|
|2021-12-10| TSLA|1008.75|1020.98|1017.03|  1017.03|        0.05|
|2021-12-09| TSLA|1060.64|1062.49| 1003.8|   1003.8|      -0.013|
|2021-12-08| TSLA|1052.71|1072.38|1068.96|  1068.96|       0.061|
|2021-12-0

On note sur le haut des données une correction du cours de TSLA

## Statistiques des actions

Pour la suite du travail, on va se concentrer sur l'action AMZN.

Toutes les fonctions sont paramétrées et il suffit de changer le paramètre "action" pour analyser d'autres actions.

In [0]:
def statistiques(action, print_list = False):
    
    """
    Cette fonction permet de générer des statistiques d'une entreprise
    
    
    Parameters
    ----------
    action : str
        Nom du titre
    print_list : bool, , default=False
        Affiche ou non la liste des statistiques de base
    returns
    -----
        Des statistiques au format json
    """
    ## Stats
    url_stats = "https://finance.yahoo.com/quote/{}/key-statistics?p={}"
    print("--Statistiques de l'entreprise "+action+ "--")
    stock = action
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    response = requests.get(url_stats.format(stock, stock), headers = headers_)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    start = script_data.find("context") - 2
    json_data = json.loads(script_data[start:-12])
    #json_data['context'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["defaultKeyStatistics"].keys()
    mylist = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["defaultKeyStatistics"]

    if print_list == True:
        print(mylist)
        
    d = pd.DataFrame(json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["defaultKeyStatistics"])
    d = d[d.columns[d.isna().sum()<3]].replace(to_replace = np.NaN, value = "-")

    j = d.to_json(orient="columns")
    parsed = json.loads(j)
    print(json.dumps(parsed, indent=2))

    return 

statistiques("AMZN", print_list = False)

--Statistiques de l'entreprise AMZN--
{
  "enterpriseToRevenue": {
    "raw": 3.914,
    "fmt": "3.91",
    "longFmt": "-"
  },
  "profitMargins": {
    "raw": 0.057350002,
    "fmt": "5.74%",
    "longFmt": "-"
  },
  "enterpriseToEbitda": {
    "raw": 29.673,
    "fmt": "29.67",
    "longFmt": "-"
  },
  "52WeekChange": {
    "raw": 0.04184282,
    "fmt": "4.18%",
    "longFmt": "-"
  },
  "forwardEps": {
    "raw": 51.59,
    "fmt": "51.59",
    "longFmt": "-"
  },
  "sharesOutstanding": {
    "raw": 507148000,
    "fmt": "507.15M",
    "longFmt": "507,148,000"
  },
  "bookValue": {
    "raw": 237.799,
    "fmt": "237.80",
    "longFmt": "-"
  },
  "sharesShort": {
    "raw": 4680878,
    "fmt": "4.68M",
    "longFmt": "4,680,878"
  },
  "sharesPercentSharesOut": {
    "raw": 0.0092,
    "fmt": "0.92%",
    "longFmt": "-"
  },
  "lastFiscalYearEnd": {
    "raw": 1609372800,
    "fmt": "2020-12-31",
    "longFmt": "-"
  },
  "heldPercentInstitutions": {
    "raw": 0.59254,
    "fmt":

## Statistiques financières

In [0]:
def financial_inf(action):

    """
    Cette fonction permet de générer des données financières d'une entreprise
    

    Parameters
    ----------
    action : str
        Nom du titre
    returns
    -----
        Les données de revenu, de la balance comptable et du cash flow au format annuel et trimestriel au format json
    """
    url_fin = "https://finance.yahoo.com/quote/{}/financials?p={}"
    stock = action
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    response = requests.get(url_fin.format(stock, stock), headers = headers_)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    #script_data[:500]
    #script_data[-500:]
    start = script_data.find("context") - 2

    json_data = json.loads(script_data[start:-12])
    #json_data['context'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

    annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["incomeStatementHistory"]["incomeStatementHistory"]
    #print(annual_is[0])
    #annual_is[0]["minorityInterest"]
    annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["cashflowStatementHistory"]["cashflowStatements"]
    annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["balanceSheetHistory"]["balanceSheetStatements"]

    quaterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["incomeStatementHistoryQuarterly"]["incomeStatementHistory"]
    quaterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["cashflowStatementHistoryQuarterly"]["cashflowStatements"]
    quaterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["balanceSheetHistoryQuarterly"]["balanceSheetStatements"]

    return annual_is, annual_cf, annual_bs, quaterly_is, quaterly_cf, quaterly_bs

annual_is, annual_cf, annual_bs, quaterly_is, quaterly_cf, quaterly_bs = financial_inf("AMZN")

In [0]:
def income_stats():
  annual_is_stmts = []
  #annual
  for s in annual_is:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    annual_is_stmts.append(statement)
  
    #quaterly
  quaterly_is_stmts = []
  for s in quaterly_is:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    quaterly_is_stmts.append(statement)


  return annual_is_stmts, quaterly_is_stmts

annual_is_stmts, quaterly_is_stmts = income_stats()

In [0]:
annual_is_stmts

Out[131]: [{'researchDevelopment': 42740000000,
  'incomeBeforeTax': 24194000000,
  'netIncome': 21331000000,
  'sellingGeneralAdministrative': 87193000000,
  'grossProfit': 152757000000,
  'ebit': 22899000000,
  'endDate': 1609372800,
  'operatingIncome': 22899000000,
  'otherOperatingExpenses': -75000000,
  'interestExpense': -1647000000,
  'incomeTaxExpense': 2863000000,
  'totalRevenue': 386064000000,
  'totalOperatingExpenses': 363165000000,
  'costOfRevenue': 233307000000,
  'totalOtherIncomeExpenseNet': 1295000000,
  'netIncomeFromContinuingOps': 21331000000,
  'netIncomeApplicableToCommonShares': 21331000000},
 {'researchDevelopment': 35931000000,
  'incomeBeforeTax': 13962000000,
  'netIncome': 11588000000,
  'sellingGeneralAdministrative': 64313000000,
  'grossProfit': 114986000000,
  'ebit': 14404000000,
  'endDate': 1577750400,
  'operatingIncome': 14404000000,
  'otherOperatingExpenses': 338000000,
  'interestExpense': -1600000000,
  'incomeTaxExpense': 2374000000,
  'tota

In [0]:
pd.DataFrame(quaterly_is_stmts)

,researchDevelopment,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
0,12097000000,4311000000,3156000000,30944000000,47882000000,4852000000,1632960000,4852000000,-11000000,-493000000,1155000000,110812000000,105960000000,62930000000,-541000000,3156000000,3156000000
1,11794000000,8646000000,7778000000,29397000000,48904000000,7702000000,1625011200,7702000000,11000000,-435000000,868000000,113080000000,105378000000,64176000000,944000000,7778000000,7778000000
2,10584000000,10263000000,8107000000,26628000000,46115000000,8865000000,1617148800,8865000000,38000000,-399000000,2156000000,108518000000,99653000000,62403000000,1398000000,8107000000,8107000000
3,16466000000,7787000000,7222000000,23428000000,46271000000,6873000000,1609372800,6873000000,-496000000,-414000000,565000000,125555000000,118682000000,79284000000,914000000,7222000000,7222000000


In [0]:
pd.DataFrame(annual_is_stmts)

,researchDevelopment,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
0,42740000000,24194000000,21331000000,87193000000,152757000000,22899000000,1609372800,22899000000,-75000000,-1647000000,2863000000,386064000000,363165000000,233307000000,1295000000,21331000000,21331000000
1,35931000000,13962000000,11588000000,64313000000,114986000000,14404000000,1577750400,14404000000,338000000,-1600000000,2374000000,280522000000,266118000000,165536000000,-442000000,11588000000,11588000000
2,28837000000,11270000000,10073000000,52177000000,93731000000,12421000000,1546214400,12421000000,296000000,-1417000000,1197000000,232887000000,220466000000,139156000000,-1151000000,10073000000,10073000000
3,22620000000,3802000000,3033000000,38992000000,65932000000,4106000000,1514678400,4106000000,214000000,-848000000,769000000,177866000000,173760000000,111934000000,-304000000,3033000000,3033000000


In [0]:
def clash_flow():
  annual_cf_stmts = []
  #annual
  for s in annual_cf:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    annual_cf_stmts.append(statement)

    #quaterly
  quaterly_cf_stmts = []
  for s in quaterly_cf:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    quaterly_cf_stmts.append(statement)


    return annual_cf_stmts, quaterly_cf_stmts
    
annual_cf_stmts, quaterly_cf_stmts = clash_flow()
annual_cf_stmts

Out[132]: [{'investments': -22242000000,
  'changeToLiabilities': 18745000000,
  'totalCashflowsFromInvestingActivities': -59611000000,
  'netBorrowings': -1104000000,
  'totalCashFromFinancingActivities': -1104000000,
  'changeToOperatingActivities': 5754000000,
  'netIncome': 21331000000,
  'changeInCash': 5967000000,
  'endDate': 1609372800,
  'effectOfExchangeRate': 618000000,
  'totalCashFromOperatingActivities': 66064000000,
  'depreciation': 25251000000,
  'changeToInventory': -2849000000,
  'changeToAccountReceivables': -8169000000,
  'changeToNetincome': 6001000000,
  'capitalExpenditures': -40140000000},
 {'investments': -9131000000,
  'changeToLiabilities': 9904000000,
  'totalCashflowsFromInvestingActivities': -24281000000,
  'netBorrowings': -10066000000,
  'totalCashFromFinancingActivities': -10066000000,
  'changeToOperatingActivities': -1383000000,
  'netIncome': 11588000000,
  'changeInCash': 4237000000,
  'endDate': 1577750400,
  'effectOfExchangeRate': 70000000,
  't

In [0]:
def balance_sheet():
  annual_bs_stmts = []
  #annual
  for s in annual_bs:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    annual_bs_stmts.append(statement)

    #quaterly
  quaterly_bs_stmts = []
  for s in quaterly_bs:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    quaterly_bs_stmts.append(statement)


    return annual_bs_stmts, quaterly_bs_stmts
    
annual_bs_stmts, quaterly_bs_stmts = balance_sheet()
annual_bs_stmts

Out[133]: [{'intangibleAssets': 4981000000,
  'capitalSurplus': 42865000000,
  'totalLiab': 227791000000,
  'totalStockholderEquity': 93404000000,
  'otherCurrentLiab': 15267000000,
  'totalAssets': 321195000000,
  'endDate': 1609372800,
  'commonStock': 5000000,
  'otherCurrentAssets': 233000000,
  'retainedEarnings': 52551000000,
  'otherLiab': 17017000000,
  'goodWill': 15017000000,
  'treasuryStock': -2017000000,
  'otherAssets': 12097000000,
  'cash': 42122000000,
  'totalCurrentLiabilities': 126385000000,
  'shortLongTermDebt': 1155000000,
  'otherStockholderEquity': -180000000,
  'propertyPlantEquipment': 150667000000,
  'totalCurrentAssets': 132733000000,
  'longTermInvestments': 5700000000,
  'netTangibleAssets': 73406000000,
  'shortTermInvestments': 42274000000,
  'netReceivables': 24309000000,
  'longTermDebt': 31816000000,
  'inventory': 23795000000,
  'accountsPayable': 72539000000},
 {'intangibleAssets': 4049000000,
  'capitalSurplus': 33658000000,
  'totalLiab': 1631880

##Modélisations avec données historiques de AMAZON

In [0]:
df, url = WebScrapStock('AMZN', days_back=360*15).get_stock()
dff = df.copy()

stock = re.findall(r'[A-Z]+', url)[0]
period1 = re.findall(r'\d{5,}', url)[0]
period2 = re.findall(r'\d{5,}', url)[1]

def lien(stock, period1, period2):
    lien = "https://finance.yahoo.com/quote/"+stock+"/history?period1="+period1+"&period2="+period2+"&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
    return lien
lien = lien(stock, period1, period2)
print("Vous pouvez voir les données çà cette adresse : ", lien)

Vous pouvez voir les données çà cette adresse :  https://finance.yahoo.com/quote/AMZN/history?period1=1173998566&period2=1640558566&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true


In [0]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3723.000000,3723.000000,3723.000000,3723.000000,3723.000000,3.723000e+03
mean,875.846866,885.119215,865.605837,875.642060,875.642060,5.433329e+06
std,1012.122049,1022.799349,1000.221283,1011.499830,1011.499830,4.446530e+06
min,35.290001,38.080002,34.680000,35.029999,35.029999,8.813000e+05
25%,166.870002,169.455002,164.815002,167.790001,167.790001,3.012600e+06
50%,341.760010,344.359985,337.160004,340.019989,340.019989,4.311700e+06
75%,1525.510010,1541.559998,1498.179993,1517.294983,1517.294983,6.511750e+06
max,3744.000000,3773.080078,3696.790039,3731.409912,3731.409912,1.043292e+08


In [0]:
df.sort_values("Date", ascending=False)

,Date,Stock,Open,High,Low,Close,Adj Close,Volume
3722,2021-12-23,AMZN,3408.560059,3439.500000,3403.000000,3421.370117,3421.370117,1837500
3721,2021-12-22,AMZN,3385.399902,3441.000000,3370.010010,3420.739990,3420.739990,2751800
3720,2021-12-21,AMZN,3357.010010,3414.330078,3312.949951,3408.340088,3408.340088,2797800
3719,2021-12-20,AMZN,3337.000000,3357.489990,3312.000000,3341.580078,3341.580078,2868600
3718,2021-12-17,AMZN,3354.209961,3417.969971,3312.270020,3400.350098,3400.350098,4277100
...,...,...,...,...,...,...,...,...
4,2007-03-21,AMZN,38.549999,39.799999,38.310001,39.799999,39.799999,4996400
3,2007-03-20,AMZN,38.529999,38.689999,38.230000,38.580002,38.580002,3801300
2,2007-03-19,AMZN,38.000000,38.540001,38.000000,38.450001,38.450001,4226600
1,2007-03-16,AMZN,37.720001,38.080002,37.520000,37.849998,37.849998,6865500


In [0]:
df.shape

Out[138]: (3723, 8)

In [0]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(
x = df4["Date"],
y = df4["Close"],
mode = "lines",
name = "Cours de fermeture AMZN ",
marker = dict(color = 'black'))


fig.add_trace(trace1, secondary_y=False)


fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='Red', rangeslider_visible=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black');
data = [trace1]

layout = dict(autosize=False,
                  width=1000,
                  height=550,
                  title = "Évolution du cours de l'action " +str(df.iloc[0, 1])+ " depuis " + str(dff.iloc[0, 0])[:10],
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de l'action",ticklen= 5, zeroline= True, showline=True, showgrid=False))
fig = dict(data = data, layout = layout)
#fig.update_xaxes()
iplot(fig)


In [0]:
df.set_index('Date', inplace=True)

Acheter des actions AMZN semble être une bonne idée

In [0]:
def preprocessing():
    
      """
        Cette fonction permet de transformer les données au format voulu

        returns
        -----
        Les inputs du modèle 
      """

      #X = df.drop(["Close", ])
      data = df.filter(["Close"])
      data_arr = data.values
      train_len = math.ceil(len(data_arr)*0.8)
      #print("Nombre d'observations des données d'entrainement", train_len)
      scaler = MinMaxScaler(feature_range=(0,1))
      data_scaled = scaler.fit_transform(data_arr)
      train = data_scaled[0:train_len, :]

      X_train = []
      y_train = []

      for i in range(60, len(train)):
        X_train.append(train[ i - 60 : i, 0])
        y_train.append(train[i, 0])

      #Conversion en array
      X_train, y_train = np.array(X_train), np.array(y_train)

      #Train set
      #reshape
      #On veut des 3D or on a des 2D
      print("--Train--")
      print(X_train.shape, "avant reshape")
      X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
      print(X_train.shape, "après reshape")

      #Test set

      test = data_scaled[train_len - 60 : , : ]
      X_test = []
      y_test = data_arr[train_len:, :]

      for i in range(60, len(test)):
        X_test.append(test[i - 60 : i, 0])

      #Conversion en array puis en 3D
      X_test = np.array(X_test)
      print("--Test--")
      print(X_test.shape, "avant reshape")
      X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
      print(X_test.shape, "après reshape")

      return X_train, y_train, X_test, y_test, train_len, scaler, data

X_train, y_train, X_test, y_test, train_len, scaler, data = preprocessing()

--Train--
(2919, 60) avant reshape
(2919, 60, 1) après reshape
--Test--
(744, 60) avant reshape
(744, 60, 1) après reshape


In [0]:
def modelisation():
      #LSTM ARN
    """
        Cette fonction entraine un modèle RNN afin de faire de la simulation in-sample des données du titre choisi 

        returns
        -----
        Les prédictions et le modèle entrainé
    """


    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape = (X_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(100))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=1, epochs = 1)

    #Predictions
    y_pred = model.predict(X_test)
    y_pred = scaler.inverse_transform(y_pred)

    #metrics
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    print("RMSE du modèle", rmse)

    return y_pred, model

y_pred, model = modelisation()

2919/2919 [==============================] - 120s 40ms/step - loss: 4.9615e-04
RMSE du modèle 80.9879018388652


In [0]:
tr = data[:train_len]
val = data[train_len:]
val["Pred"] = y_pred
val = val.reset_index()
tr = tr.reset_index()

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(
x = tr["Date"],
y = tr["Close"],
mode = "lines",
name = "Données de d'entrainement",
marker = dict(color = 'black'))

trace2 = go.Scatter(
x = val["Date"],
y = val["Close"],
mode = "lines",
name = "Données de test",
marker = dict(color = 'blue'))

trace3 = go.Scatter(
x = val["Date"],
y = val["Pred"],
mode = "lines",
name = "données prédites",
marker = dict(color = '#CB7272'))
fig2.add_trace(trace2, secondary_y=False)
fig2.add_trace(trace3, secondary_y=False)

fig2.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='Red', rangeslider_visible=True)
fig2.update_yaxes( showline=True, linewidth=2, linecolor='black', gridcolor='black');

data = [trace1, trace2, trace3]
data1 = [trace2, trace3]
layout = dict(autosize=False,
                  width=1200,
                  height=550,
                  title = "Évolution du cours de l'action "+ str(dff.iloc[0, 1])+ " depuis " + str(dff.iloc[0, 0])[:10],
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de l'action",ticklen= 5, zeroline= True, showline=True, showgrid=False))

layout1 = dict(autosize=False,
                  width=1200,
                  height=550,
                  title = "Zoom sur la période de test",
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de l'action",ticklen= 5, zeroline= True, showline=True, showgrid=False))
fig2 = dict(data = data, layout = layout)
fig3 = dict(data = data1, layout = layout1)
iplot(fig2)
iplot(fig3)


In [0]:
def predict_next_val():
  
    """
        Cette fonction permet de prédire la valeu du titre le prochain jour de notation

        returns
        -----
        None
    """
    df1_close = df.filter(["Close"])

    last_60 = df1_close[-60:].values
    last_60_scl = scaler.transform(last_60)

    x_test = []
    x_test.append(last_60_scl)
    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    pred = model.predict(x_test)
    pred = scaler.inverse_transform(pred)

    print("La valeur de l'action pour le prochain jour de cotation est",pred[0][0], "$ en sachant qu'il est côté actuellement %.3f" %dff.iloc[-1, 5], "$")

    return 
predict_next_val()

La valeur de l'action pour le prochain jour de cotation est 3342.0098 $ en sachant qu'il est côté actuellement 3421.370 $
